# Ifremer prototype: balast

$
\begin{align}
m_{f,water} &=  (\rho-\rho_w)V \\
m_{f+m,water} &= m_{m,water} + (\rho-\rho_w)V, \\
&= m_{m,water} + m - \rho_w V,
\end{align}
$

where $m_{f,water}$ and $m_{m,water}$ are the weights in water of the float and the additional mass respectively, and, where $\rho = m/V$ is the nominal density float, $m$ is the mass of the float (in air).

$
\begin{align}
\delta m_{f,water} = 0 &= \delta m - V \delta \rho_w - \rho_w \delta V,
\end{align}
$

--- 

**Balasting protocol:**

1. weigh the float in air

2. set piston to target position: big piston inside, small piston a fraction (e.g. 1/4) inside.

3. put the float in the tank and let it adjust to the temperature (e.g. 15min? need to find appropriate value)

4. weigh the float + an additional mass: $m_{f+m, water}$. The float needs to be fully underwater.

5. weigh the additional mass in water: $m_{m, water}$

6. deduce the weight of the float in water: $m_{f, water} = m_{f+m, water} - m_{m, water}$

7. measure temperature, salinity (or conductivity) in the tank

**Checklist for tests:**

1. Bring: weights labelled with weight in water, tape, rope, CTD (with computer and accessories)

2. measure water temperature, salinity (or conductivity) in the deployment area and at the surface.

3. adjust mass according to computations below


---

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import gsw

import cognac.float as fl

---

## first deployment: lopstech19 leg 2

In [2]:
bfile = '/Users/aponte/Current_projects/cognac/campagnes_techno/2019_lopstech/leg2/float/balast.yaml'

### generate balast file

In [2]:
#
b = fl.balast()

In [3]:
b.add_balast('ifremer-15/10/2019', mass_in_water=0., mass_in_air=13554., 
             water_temperature=17.4, water_salinity=34.622, 
             piston_displacement=9.*3./4.)
#
b.store(bfile)

Balasting data stored in /Users/aponte/Current_projects/cognac/campagnes_techno/2019_lopstech/leg2/float/balast.yaml


### reload balast file

In [3]:
b = fl.balast(file=bfile)

File /Users/aponte/Current_projects/cognac/campagnes_techno/2019_lopstech/leg2/float/balast.yaml loaded


In [4]:
b

{ 'ifremer-15/10/2019': { 'CT': 17.40100549537287,
                          'SA': 34.78738585389333,
                          'V': 0.013221626276530922,
                          'comments': 'None',
                          'mass_in_air': 13554.0,
                          'mass_in_water': 0.0,
                          'piston_displacement': 6.75,
                          'rho_water': 1025.1386415345182,
                          'water_salinity': 34.622,
                          'water_temperature': 17.4}}

In [5]:
# init float
f = fl.autonomous_float(model='Ifremer')
f.init_piston()

Piston max volume set from max displacement


---

### compute adjustments

In [18]:
# use 1 point to compute mass adjustment
# could think also about loading tsg data
#b.compute_mass_adjustment(f, temperature=19.4, salinity=37.99, lon='5E23.634', lat='43N02.757')
#b.compute_mass_adjustment(f, temperature=19.97, salinity=37.82, lon='5E26.758', lat='43N00.910')
b.compute_mass_adjustment(f, temperature=19.824, salinity=37.85, lon='5E29.328', lat='43N00.390') # d0
# with this last balast, the float was sinking at nbpulse_gpiston (14??). 
#    but with 20g off, it was floating.
b.compute_mass_adjustment(f, temperature=18.643, salinity=38.021, lon='5E34.638', lat='42N53.959') # d1

According to balast ifremer-15/10/2019, you need add 43.3 g
According to balast ifremer-15/10/2019, you need add 48.0 g


In [ ]:
# use water profile to compute mass adjustement
w = fl.waterp(lon=6.,lat=40.) # Mediterranean sea
print(w)
b.compute_mass_adjustment(f, w)

---

# 2019/11: tests ifremer

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import gsw

import cognac.float as fl

In [2]:
bfile = '/Users/aponte/Current_projects/cognac/instrum/flotteur/tests/balast_201911.yaml'

In [3]:
#
b = fl.balast()

In [4]:
b.add_balast('ifremer-18/11/2019', mass_in_water=-121., mass_in_air=13413., 
             water_temperature=18.2706, water_conductivity=44.6036, 
             piston_displacement=0.)
#
b.store(bfile)

Practical salinity derived from conductivity: SP=33.67
Balasting data stored in /Users/aponte/Current_projects/cognac/instrum/flotteur/tests/balast_201911.yaml


In [3]:
b = fl.balast(file=bfile)

File /Users/aponte/Current_projects/cognac/instrum/flotteur/tests/balast_201911.yaml loaded


In [4]:
b

{ 'ifremer-18/11/2019': { 'CT': 18.300396039190872,
                          'SA': 33.83300763196808,
                          'V': 0.01309608458116599,
                          'comments': 'None',
                          'mass_in_air': 13413.0,
                          'mass_in_water': -121.0,
                          'piston_displacement': 0.0,
                          'rho_water': 1024.199249544385,
                          'water_salinity': 33.672159073801225,
                          'water_temperature': 18.2706}}

In [5]:
# init float
f = fl.autonomous_float(model='Ifremer')
f.init_piston()

Piston max volume set from max displacement


In [6]:
# use 1 point to compute mass adjustment
b.compute_mass_adjustment(f, temperature=12.99, conductivity=42.5, 
                          lon=b.lon, lat=b.lat, verbose=True)

-- According to balast ifremer-18/11/2019, you need add 159.9 g
(if the weight is external to the float, remember this must be the value in water)
Independent mass correction (balast -mass_in_water): 121.0 [g]
  Water change mass correction: 38.9 [g]
    T contribution: 14.3 [g]
    S contribution: 28.2 [g]
  New/old in situ density: 1027.55, 1024.20  [kg/m^3] 
    difference: 3.4 [kg/m^3]
  New/old float volume: 1.31e-02, 1.31e-02  [m^3] 
    difference: -4.8 [cm^3]


**Observed behavior**:
- with 117.5g (in water) the float does not sink even with the piston fully in (+30g approx)
- with 128.5g (in water) the float is able to sink with the piston fully in (+30g approx)

---
## other interesting values

In [8]:
# change of mass in g after 1deg temperature change
dT = 1.
dv = f.alpha*f.m * dT
dv*1e3 # g

0.9460692

In [9]:
print('Float thermal expansion = %1.2e' %(f.alpha))
alpha, beta = w.get_alpha(-1.), w.get_beta(-1.)
print('Water: thermal expansion = %1.2e and haline contraction =% 1.2e' %(alpha, beta))

dT, dS = 1., 1.
for name, _b in b._d.items():
    print('%.1f deg float temperature change leads to an equivalent mass change of %.2f g' %(dT, f.alpha*f.m*dT*1e3))
    print('%.1f deg water temperature change leads to an equivalent mass change of %.2f g' %(dT,alpha*f.m*dT*1e3))
    print('%.1f g/kg salinity change leads to an equivalent mass change of %.2f g' %(dS,beta*f.m*dS*1e3))

Float thermal expansion = 6.98e-05
Water: thermal expansion = 2.57e-04 and haline contraction = 7.33e-04
1.0 deg change leads to a float equivalent mass change of 0.95 g
1.0 deg change leads to an equivalent mass change of 3.48 g
1.0 g/kg salinity change leads to an equivalent mass change of 9.93 g


In [10]:
# miscellaneous
f.v*1e6 # volume out in cm3

20.15871648515188

In [11]:
f

Float parameters: 
  L     = 0.83 m      - float length
  r     = 0.07 m      - float radius
  m     = 13.55 kg     - float mass
  V     = 1.32e+04 cm^3   - float volume
  rho_cte = m/V = 9.13e+08 kg.cm^3   - float baseline density
  gamma = 3.78e-06 /dbar  - mechanical compressibility
  alpha = 6.98e-05 /degC  - thermal compressibility
  temp0 = 0.00e+00  degC  - reference temperature
  a = 1.00e+00  (no dimension)  - float added mass
  c0 = 0.00e+00  (no dimension)  - float drag parameter 0
  c1 = 1.00e+00  (no dimension)  - float drag parameter 1
Piston parameters and state: 
  r     = 9.75 mm        - piston radius
  phi   = 0.42 rad       - present angle of rotation
  d     = 67.50 mm        - present piston displacement
  vol   = 20.16 cm^3      - present volume addition
  lead  = 100.00 cm        - screw lead
  d_increment  = 0.00 m        - smallest variation of translation motion for the piston
  vol_error  = 6.40e-08 m^3        - smallest variation of volume possible for the 

---

### lead sheets weight in water / weight in air:

In [10]:
print(30/41)
print(19/25)
print(20/28)

0.7317073170731707
0.76
0.7142857142857143


In [12]:
15 * 0.73

10.95

---

https://webmail.ifremer.fr/?_task=mail&_mbox=BALISES_NOVATECH

login:  planeurs

pwd: caplane4

https://cas.ifremer.fr/cas/login?service=http%3a%2f%2ftriagoz.ifremer.fr%2fLOPStrack

login pwd intranet ifremer

300434063369340

300434063262410

